In [4]:
import os 
import numpy as np
import pandas as pd
import mne 
import h5py
import matplotlib.pyplot as plt
from PIL import Image


In [6]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

base_path = '/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/_IBLworkflows/decoding/figures'
base_TF_path = '/mnt/data/AdaptiveControl/IBLrawdata/TF_data'
conditions = ['Stim_NoStim', 'Right_Left', 'BiasRight_BiasLeft']
version = 'csd'

def plot_images(pid):

    path_fig = os.path.join(base_path, pid)
    if not os.path.isdir(path_fig):
        print(f"Skipping {pid}")
        return
    for condition in conditions:
        
      
        files = [f for f in os.listdir(path_fig) if condition in f and version in f]
        files = sorted(files)
        
        if not files:
            return
        
        
        num_files = len(files)
        num_columns = 3
        num_rows = (num_files + num_columns - 1) // num_columns  # calculate the number of rows needed
        
        fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))
        axes = axes.flatten()
        
        for ax, file in zip(axes, files):
            img_path = os.path.join(path_fig, file)
            img = mpimg.imread(img_path)
            ax.imshow(img)
            ax.set_title(file)
            ax.axis('off')
        
        for ax in axes[num_files:]:
            fig.delaxes(ax)
        
        plt.tight_layout()
        output_filename = f"{condition}_{version}_{pid}.png"
        output_filepath = os.path.join(base_path, output_filename)
        # plt.show()
        plt.savefig(output_filepath)
        plt.close()


In [7]:
import submitit

base_TF_path = '/mnt/data/AdaptiveControl/IBLrawdata/TF_data'

for pid in os.listdir(base_TF_path):
    executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
    executor.update_parameters(mem_gb=4, timeout_min=300, slurm_partition="CPU", cpus_per_task=1, slurm_comment='compute_epochTFR')
    job = executor.submit(plot_images, pid)